#运行一下倒入模块的包

In [1]:
import pandas as pd
import os
import numpy as np
from gensim.models import KeyedVectors
from sklearn.utils import shuffle
import jieba

from tensorflow.keras.layers import Input, Dense, LSTM, Conv1D,Dropout,Bidirectional,Multiply,Embedding,MaxPooling1D,ConvLSTM2D,Conv2D,MaxPooling2D
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
# below bag may have different path 
from tensorflow.python.keras.layers.merge import concatenate  
from tensorflow.python.keras.layers.core import *
from tensorflow.python.keras.layers.recurrent import LSTM
from tensorflow.keras.models import *
from tensorflow.keras.utils import to_categorical

In [2]:
## load embedding
wv_from_text = KeyedVectors.load_word2vec_format('./tencent-ailab-embedding-zh-d100-v0.2.0-s/tencent-ailab-embedding-zh-d100-v0.2.0-s.txt', binary=False)

In [32]:
## load training data
file_name = "starbuck_2018_tag"
pd_all = pd.read_csv(file_name+'.csv')
pd_all = shuffle(pd_all)
x_data = np.array(pd_all['正文'])
y_data = np.array(pd_all['sentimate'])

In [33]:
y_data[np.where(y_data<=0.5)[0]]=0
y_data[np.where(y_data>0.5)[0]]=1

In [7]:
## global factor
CONTENT_LEN = 50
# INPUT_DIMS = wv_from_text.vector_size
INPUT_DIMS = 100
TIME_STEPS = CONTENT_LEN
lstm_units = 64
bs = 100

In [6]:
## Tokenize word
with open('stopwords.txt','r',encoding='utf-8') as f:
    stopwords = f.readlines()
    stopwords = [x.rstrip() for x in stopwords]


In [34]:
## padding the sentence
all_content = []
for i in range(x_data.shape[0]):
    content_i = jieba.lcut(x_data[i])
    ## cut text or padding
    if len(content_i) > CONTENT_LEN:
        content_i = content_i[:CONTENT_LEN]
    elif len(content_i) < CONTENT_LEN:
        for _ in range(CONTENT_LEN-len(content_i)):
            content_i.append('</s>')
    ## add current review to all_content(n_observations,sentence_length)
    all_content.append(content_i)


In [9]:
m = load_model(filepath='./model.h5')
m.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50, 100)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 50, 64)       6464        ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 50, 64)       0           ['conv1d[0][0]']                 
                                                                                                  
 bidirectional (Bidirectional)  (None, 50, 128)      66048       ['dropout[0][0]']                
                                                                                              

In [35]:
pd_all['tag']=pd.Series() 

<ipython-input-35-0bafd78a265e>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd_all['tag']=pd.Series()


In [36]:
for k in range(bs):
    index_start = len(all_content)//bs*k
    index_end = len(all_content)//bs*(k+1)
    batch_content = all_content[index_start:index_end]
    b_y_data = y_data[index_start:index_end]
    b_x_data = x_data[index_start:index_end]

    ## vectorize each word
    v_a_content = np.empty((len(batch_content),CONTENT_LEN,wv_from_text.vector_size),dtype='float')
    for i in range(len(batch_content)):
        for j in range(CONTENT_LEN):
            ## if current word in ailab, convert to vector
            try:
                v_a_content[i][j] = list(wv_from_text.get_vector(batch_content[i][j]))
            ## otherwise, add the first char's vector
            except:
                v_a_content[i][j] = list(wv_from_text.get_vector('</s>'))

    # v_a_content.shape
    y = to_categorical(b_y_data)
    preds = m.predict(v_a_content)
    pd_all['tag'][index_start:index_end]=preds.argmax(axis=1)
    # m.fit(v_a_content,y, epochs=5,batch_size=64, validation_split=0.1)


44/44 [==============================] - 1s 14ms/step


<ipython-input-36-dbc8bc7ebd43>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_all['tag'][index_start:index_end]=preds.argmax(axis=1)


44/44 [==============================] - 1s 14ms/step


In [37]:
pd_all.to_csv(file_name+'_new.csv')

In [38]:
!pip install snownlp

  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760960 sha256=67fab21b49c9e15b8cb3f1dcdc68f46a23dfd49e73d5aacd0d4ae9ea8039c4b2
  Stored in directory: c:\users\74460\appdata\local\pip\cache\wheels\09\14\c5\ea9aee34229caa97c6f6ff78c82c7f2b1a3423c1f37227b6a6
Successfully built snownlp


## evaluate LSTM with snowNLP

In [39]:
from snownlp import SnowNLP 

In [42]:
text = pd.read_csv('weibo_senti_100k.csv')

In [43]:
text.head()

,label,review
0,1,﻿更博了，爆照了，帅的呀，就是越来越爱你！生快傻缺[爱你][爱你][爱你]
1,1,@张晓鹏jonathan 土耳其的事要认真对待[哈哈]，否则直接开除。@丁丁看世界 很是细心...
2,1,姑娘都羡慕你呢…还有招财猫高兴……//@爱在蔓延-JC:[哈哈]小学徒一枚，等着明天见您呢/...
3,1,美~~~~~[爱你]
4,1,梦想有多大，舞台就有多大![鼓掌]


In [45]:
text['sentimate']=pd.Series() 
for i in range(text.shape[0]):
    s = SnowNLP(text['review'][i])
    text['sentimate'][i]=s.sentiments

<ipython-input-45-22f13e32a990>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  text['sentimate']=pd.Series()
<ipython-input-45-22f13e32a990>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['sentimate'][i]=s.sentiments


In [46]:
x_data = np.array(text['review'])
y_data = np.array(text['sentimate'])
true_label = np.array(text['label'])

In [47]:
y_data[np.where(y_data<=0.5)[0]]=0
y_data[np.where(y_data>0.5)[0]]=1

In [48]:
## padding the sentence
all_content = []
for i in range(x_data.shape[0]):
    content_i = jieba.lcut(x_data[i])
    ## cut text or padding
    if len(content_i) > CONTENT_LEN:
        content_i = content_i[:CONTENT_LEN]
    elif len(content_i) < CONTENT_LEN:
        for _ in range(CONTENT_LEN-len(content_i)):
            content_i.append('</s>')
    ## add current review to all_content(n_observations,sentence_length)
    all_content.append(content_i)

In [49]:
text['tag']=pd.Series() 

<ipython-input-49-e1232b41aa33>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  text['tag']=pd.Series()


In [50]:
for k in range(bs):
    index_start = len(all_content)//bs*k
    index_end = len(all_content)//bs*(k+1)
    batch_content = all_content[index_start:index_end]
    b_y_data = y_data[index_start:index_end]
    b_x_data = x_data[index_start:index_end]

    ## vectorize each word
    v_a_content = np.empty((len(batch_content),CONTENT_LEN,wv_from_text.vector_size),dtype='float')
    for i in range(len(batch_content)):
        for j in range(CONTENT_LEN):
            ## if current word in ailab, convert to vector
            try:
                v_a_content[i][j] = list(wv_from_text.get_vector(batch_content[i][j]))
            ## otherwise, add the first char's vector
            except:
                v_a_content[i][j] = list(wv_from_text.get_vector('</s>'))

    # v_a_content.shape
    y = to_categorical(b_y_data)
    preds = m.predict(v_a_content)
    text['tag'][index_start:index_end]=preds.argmax(axis=1)

38/38 [==============================] - 1s 14ms/step


<ipython-input-50-cce944d67058>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['tag'][index_start:index_end]=preds.argmax(axis=1)


38/38 [==============================] - 1s 13ms/step


In [55]:
k

99

In [56]:
batch_content = all_content[index_end:]
b_y_data = y_data[index_end:]
b_x_data = x_data[index_end:]

## vectorize each word
v_a_content = np.empty((len(batch_content),CONTENT_LEN,wv_from_text.vector_size),dtype='float')
for i in range(len(batch_content)):
    for j in range(CONTENT_LEN):
        ## if current word in ailab, convert to vector
        try:
            v_a_content[i][j] = list(wv_from_text.get_vector(batch_content[i][j]))
        ## otherwise, add the first char's vector
        except:
            v_a_content[i][j] = list(wv_from_text.get_vector('</s>'))

# v_a_content.shape
y = to_categorical(b_y_data)
preds = m.predict(v_a_content)
text['tag'][index_end:]=preds.argmax(axis=1)

3/3 [==============================] - 0s 15ms/step


<ipython-input-56-8ad697fda1b8>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['tag'][index_end:]=preds.argmax(axis=1)


In [51]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [59]:
pred_2 = np.c_[np.array(text['tag']),y_data]

In [60]:
pred_2

array([[1., 1.],
       [1., 1.],
       [1., 1.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [61]:
_,pred_label,_,true_label = train_test_split(pred_2,np.array(text['label']))

In [62]:
print('BiLSTM',classification_report(pred_label[:,0],true_label))

BiLSTM               precision    recall  f1-score   support

         0.0       0.94      0.94      0.94     15006
         1.0       0.94      0.94      0.94     14991

    accuracy                           0.94     29997
   macro avg       0.94      0.94      0.94     29997
weighted avg       0.94      0.94      0.94     29997



In [63]:
print('snowNLP',classification_report(pred_label[:,1],true_label))

snowNLP               precision    recall  f1-score   support

         0.0       0.32      0.68      0.43      7034
         1.0       0.85      0.55      0.67     22963

    accuracy                           0.58     29997
   macro avg       0.58      0.62      0.55     29997
weighted avg       0.73      0.58      0.61     29997

